## Running inference with `croptypemapper`

In [11]:
import sys 
import os
import inspect
root_dir = os.environ['PWD']
# sys.path.append('/home/mappers/projects/croptypemapper/croptypemapper')
# works
sys.path.insert(0, '/home/mappers/projects/croptypemapper/')
sys.path.insert(0, '/home/mappers/projects/croptypemapper/croptypemapper/')


# print(sys.path)
import torch
import torch.nn as nn
import torch.utils.data
# from torch import nn
import yaml

# os.listdir('/home/mappers/projects/croptypemapper/croptypemapper')
# os.getcwd()
import importlib
import croptypemapper
importlib.reload(croptypemapper)

import models
importlib.reload(models)
from models import *
# os.environ['PWD']

In [10]:
# sys.path.pop()
# sys.path.remove('/home/mappers/projects/croptypemapper/croptypemapper/')
# print(sys.path)
# Double_branch_stacked_biLSTM
# os.environ
# os.getlogin()
# get_inp_branch()
# print(inspect.getsource(Double_branch_stacked_biLSTM))

## Functions

In [ ]:
# def get_inp_branch(x):
#     if isinstance(x, tuple) or isinstance(x, list):
#         br1 = x[0]
#         br2 = x[1]
#     elif isinstance(x, int) or isinstance(x, float):
#         br1 = x
#         br2 = x
           
#     return br1, br2

# class Double_branch_stacked_biLSTM(torch.nn.Module):
#     def __init__(self, input_dims = (4, 11), hidden_dims = (64, 64), n_classes = 4, n_layers = (2, 2), 
#                  dropout_rate = (0.35, 0.45), s1_weight = 0.6, bidirectional = True, use_layernorm = True, 
#                  use_batchnorm = False, use_attention = False):
#         super(Double_branch_stacked_biLSTM, self).__init__()
        
#         # Define object properties
#         self.n_classes = n_classes
#         self.s1_weight = s1_weight
#         self.bidirectional = bidirectional
#         self.use_layernorm = use_layernorm
#         self.use_batchnorm = use_batchnorm
#         self.use_attention = use_attention

        
#         s1_in_dim, s2_in_dim = get_inp_branch(input_dims)
#         s1_hidden_dim, s2_hidden_dim = get_inp_branch(hidden_dims)
#         s1_n_layers, s2_n_layers = get_inp_branch(n_layers)
#         s1_dropout_rate, s2_dropout_rate = get_inp_branch(dropout_rate)
        
#         # Layer normalization for s1, s2 inputs and current_states of LSTM
#         if self.use_layernorm:
#             self.s1_inlayernorm = nn.LayerNorm(s1_in_dim)
#             self.s1_clayernorm = nn.LayerNorm((s1_hidden_dim + s1_hidden_dim * self.bidirectional) * s1_n_layers)
            
#             self.s2_inlayernorm = nn.LayerNorm(s2_in_dim)
#             self.s2_clayernorm = nn.LayerNorm((s2_hidden_dim + s2_hidden_dim * self.bidirectional) * s2_n_layers)
        
#         # LSTM layers for s1 and s2
#         self.s1_lstm = nn.LSTM(input_size = s1_in_dim, hidden_size = s1_hidden_dim, 
#                                num_layers = s1_n_layers, bias = False, batch_first = True, dropout = s1_dropout_rate, 
#                                bidirectional = self.bidirectional)
        
#         self.s2_lstm = nn.LSTM(input_size = s2_in_dim, hidden_size = s2_hidden_dim, 
#                                num_layers = s2_n_layers, bias = False, batch_first = True, dropout = s2_dropout_rate, 
#                                bidirectional = self.bidirectional)
        
#         if self.bidirectional:
#             s1_hidden_dim = s1_hidden_dim * 2
#             s2_hidden_dim = s2_hidden_dim * 2
        
#         if self.use_attention:
#             self.attention = attention()
        
#         # MLP layer on top of LSTM
#         s1_linear_input_dim = s1_hidden_dim if self.use_attention else s1_hidden_dim * s1_n_layers
#         self.s1_linear_class = nn.Linear(s1_linear_input_dim, self.n_classes, bias = True)
        
#         s2_linear_input_dim = s2_hidden_dim if self.use_attention else s2_hidden_dim * s2_n_layers
#         self.s2_linear_class = nn.Linear(s2_linear_input_dim, self.n_classes, bias = True)
 

#     def _logits(self, s1, s2):
#         #set_trace()
#         if self.use_layernorm:
#             s1 = self.s1_inlayernorm(s1)
#             s2 = self.s2_inlayernorm(s2)
        
#         # Get outputs and the last current state and hidden state for each branch.
#         #s1_outputs & s2_outputs: [B, Seq_length, 2 x hidden_dim]
#         s1_outputs, s1_last_state_list = self.s1_lstm.forward(s1)
#         s2_outputs, s2_last_state_list = self.s2_lstm.forward(s2)
        
#         #s1_h & s1_c & s2_h & s2_c: [2 x num_layers, B, hidden_dim] 
#         s1_h, s1_c = s1_last_state_list
#         s2_h, s2_c = s2_last_state_list
        
#         # Get the query layer to calculate self attention for each branch
#         if self.use_attention:
#             if self.bidirectional:
#                 # Get the last state of each branch. size:[B, hidden_dim]
#                 s1_query_forward = s1_c[-1]
#                 s1_query_backward = s1_c[-2]
#                 # size:[B, 2 x hidden_dim]
#                 s1_query = torch.cat([s1_query_forward, s1_query_backward], 1)
                
#                 s2_query_forward = s2_c[-1]
#                 s2_query_backward = s2_c[-2]
#                 s2_query = torch.cat([s2_query_forward, s2_query_backward], 1)
#             else:
#                 s1_query = s1_c[-1]
#                 s2_query = s2_c[-1]
            
#             # Get attention weights and hidden state
#             s1_h, s1_weights = self.attention(s1_query, s1_outputs, s1_outputs)
#             s2_h, s2_weights = self.attention(s2_query, s2_outputs, s2_outputs)
#             s1_h = s1_h.squeeze(1)
#             s2_h = s2_h.squeeze(1)
#         else:
#             s1_nlayers, s1_batchsize, s1_n_hidden = s1_c.shape
#             s2_nlayers, s2_batchsize, s2_n_hidden = s2_c.shape
#             s1_h = self.s1_clayernorm(s1_c.transpose(0,1).contiguous().view(s1_batchsize, s1_nlayers * s1_n_hidden))
#             s2_h = self.s2_clayernorm(s2_c.transpose(0,1).contiguous().view(s2_batchsize, s2_nlayers * s2_n_hidden))
        
#         # Calculate logits for each branch. Shape:[B, num_classes]
#         s1_logits = self.s1_linear_class.forward(s1_h)
#         s2_logits = self.s2_linear_class.forward(s2_h)
        
#         if self.use_attention:
#             s1_pts = s1_weights
#             s2_pts = s2_weights
#         else:
#             s1_pts = None
#             s2_pts = None
        
#         return s1_logits, s2_logits, s1_pts, s2_pts
    
#     def forward(self, s1, s2):
        
#         s1_logits, s2_logits, s1_pts, s2_pts = self._logits(s1, s2)
#         fused_logits = (s1_logits * self.s1_weight) + (s2_logits * (1 - self.s1_weight))
        
#         return s1_logits, s2_logits, fused_logits

## Configuration file

In [12]:
# yaml_path = '../../config-working.yaml'
# with open(yaml_path, 'r') as yaml_file:
#     config = yaml.safe_load(yaml_file)
# # config
# get_inp_branch(config["input_dims"])
# config["input_dims"]
# Double_branch_stacked_biLSTM

config = {
    
    "working_dir" : "C:/My_documents/CropTypeData_Rustowicz/working_folder",
    "out_dir": "new_try_01",
    # Dataset & Loader
    "root_dir" : "C:/My_documents/CropTypeData_Rustowicz/CropType",
    "sampling_strategy" : "ranked",
    "lbl_fldrname" : "Labels",
    "sources" : ["Sentinel-1", "Sentinel-2"],
    "num_train_pixels" : 15000,
    "num_validation_pixels" : 10000,
    "test_label" : False,
    "batch_train" : 128,
    "batch_val" : 1,
    
    # Model Compiler
    "init_params" : "/home/mappers/data/models/v1/model_path.rar",
    "gpus" : [0],
    "input_dims" : (4, 11),
    "LSTM_hidden_dim" : (48, 64),
    "CNN_hidden_dim" : (48, 64),
    "CNN_kernel_size" : (5, 5),
    "CNN_sequence_length" : (57, 67),
    "n_classes": 4,
    "n_LSTM_layers" : (2, 4),
    "LSTM_lyr_dropout_rate" : (0.4, 0.5),
    "CNN_lyr_dropout_rate" : (0.25, 0.45),
    "s1_weight" : 0.5,
    
    # Model fitting
    "epoch" : 75,
    "optimizer" : "amsgrad",
    "momentum" : 0.95,
    "criterion" : nn.CrossEntropyLoss,
    "branch_weights" : (0.3, 0,3, 0.4),
    "lr_init" : 0.01,
    "LR_policy" : "",
    
    "bucket" : None,
    "save_fldr": "model_path",
    "prefix_out": "/home/mappers/data/predictions/croptypes/v1/"
}

In [13]:
config

{'working_dir': 'C:/My_documents/CropTypeData_Rustowicz/working_folder',
 'out_dir': 'new_try_01',
 'root_dir': 'C:/My_documents/CropTypeData_Rustowicz/CropType',
 'sampling_strategy': 'ranked',
 'lbl_fldrname': 'Labels',
 'sources': ['Sentinel-1', 'Sentinel-2'],
 'num_train_pixels': 15000,
 'num_validation_pixels': 10000,
 'test_label': False,
 'batch_train': 128,
 'batch_val': 1,
 'init_params': '/home/mappers/data/models/v1/model_path.rar',
 'gpus': [0],
 'input_dims': (4, 11),
 'LSTM_hidden_dim': (48, 64),
 'CNN_hidden_dim': (48, 64),
 'CNN_kernel_size': (5, 5),
 'CNN_sequence_length': (57, 67),
 'n_classes': 4,
 'n_LSTM_layers': (2, 4),
 'LSTM_lyr_dropout_rate': (0.4, 0.5),
 'CNN_lyr_dropout_rate': (0.25, 0.45),
 's1_weight': 0.5,
 'epoch': 75,
 'optimizer': 'amsgrad',
 'momentum': 0.95,
 'criterion': torch.nn.modules.loss.CrossEntropyLoss,
 'branch_weights': (0.3, 0, 3, 0.4),
 'lr_init': 0.01,
 'LR_policy': '',
 'bucket': None,
 'save_fldr': 'model_path',
 'prefix_out': '/home/ma

## Predictions

In [ ]:
# Double_branch_stacked_biLSTM

### Set-up model

In [14]:
# Initialize the model
lstm_model = Double_branch_stacked_biLSTM(
    input_dims = config["input_dims"],
    hidden_dims = config["LSTM_hidden_dim"], 
    n_classes = config["n_classes"], 
    n_layers = config["n_LSTM_layers"], 
    dropout_rate = config["LSTM_lyr_dropout_rate"], 
    s1_weight = config["s1_weight"], 
    bidirectional = True, 
    use_layernorm = True, 
    use_batchnorm = False, 
    use_attention = False
)

In [15]:
lstm_model

Double_branch_stacked_biLSTM(
  (s1_inlayernorm): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
  (s1_clayernorm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  (s2_inlayernorm): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
  (s2_clayernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (s1_lstm): LSTM(4, 48, num_layers=2, bias=False, batch_first=True, dropout=0.4, bidirectional=True)
  (s2_lstm): LSTM(11, 64, num_layers=4, bias=False, batch_first=True, dropout=0.5, bidirectional=True)
  (s1_linear_class): Linear(in_features=192, out_features=4, bias=True)
  (s2_linear_class): Linear(in_features=512, out_features=4, bias=True)
)

In [16]:
# Compile the model
model = ModelCompiler(model=lstm_model,
                      working_dir=config["working_dir"], 
                      out_dir=config["out_dir"],
                      gpuDevices=config["gpus"],
                      br_weights = config["branch_weights"],
                      params_init=config["init_params"],
                      freeze_params=None)

NameError: name 'ModelCompiler' is not defined

### Load prediction tiles

In [17]:
def load_data_pred(usage, item):
    dataset = pixelDataset(root_dir = config["root_dir"],
                           usage = "test",
                           sources = config["sources"],
                           inference_index = item)
    tile = dataset.tile_id
    meta = dataset.meta
    data_loader = DataLoader(dataset, 
                          batch_size=config["batch_val"], 
                          shuffle = False)
    
    return data_loader, meta, tile

### Run predictions

In [22]:
import numpy as np
s1_pth = '/home/mappers/data/tiles/s1/source_s1_539059.npy'
s1_array = np.load(s1_pth)

s2_pth = '/home/mappers/data/tiles/s2/source_s2_539059.npy'
s2_array = np.load(s2_pth)

In [33]:
# s1_array[0, 0:550, 0:550, 2]
# s2_array.shape

In [34]:
# s2_array[10, 0:550, 0:550, 2]

In [ ]:
prediction_dir = Path(config[root_dir]).joinpath(
    "prediction_tiles", config["sources"][0]
)
s1_fnames = [Path(dirpath) / f for (dirpath, dirnames, filenames) \
             in os.walk(prediction_dir) for f in filenames \
             if f.endswith(".npy")]
tile_count = len(s1_fnames)

for i in range(tile_count):
    pred_data = load_data_pred("test", i)
    model.inference(pred_data, config["prefix_out"])a

## Filter predictions through field boundaries